In [1]:


from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=2024, day=int("14"))

puzzle.header()
# example = get_code_block(puzzle, 5)

# Restroom Redoubt

[Open Website](https://adventofcode.com/2024/day/14)

In [2]:
import re


class Robot:

    def __init__(self, position: tuple, velocity: tuple):
        self.position = (position[0], position[1])
        self.velocity = (velocity[0], velocity[1])

    def move(self, dimensions):
        self.position = (self.position[0] + self.velocity[0], self.position[1] + self.velocity[1])
        self.position = (self.position[0] % dimensions[0], self.position[1] % dimensions[1])

    def __str__(self):
        return f"Robot(position={self.position}, velocity={self.velocity})"


# helper functions
def domain_from_input(input: InputReader) -> list:
    # create regex that matches on 0,4 or 3,-3
    regex = re.compile(r"(-?\d+),(-?\d+)")

    robots = []

    lines = input.lines_as_strs()
    for line in lines:
        position_str = regex.findall(line[0])[0]
        velocity_str = regex.findall(line[1])[0]
        position = list(map(int, position_str))
        velocity = list(map(int, velocity_str))
        robot = Robot(position, velocity)
        robots.append(robot)

    return robots


test_input = domain_from_input(puzzle.example(0))
for robot in test_input:
    print(robot)

Robot(position=(0, 4), velocity=(3, -3))
Robot(position=(6, 3), velocity=(-1, -3))
Robot(position=(10, 3), velocity=(-1, 2))
Robot(position=(2, 0), velocity=(2, -1))
Robot(position=(0, 0), velocity=(1, 3))
Robot(position=(3, 0), velocity=(-2, -2))
Robot(position=(7, 6), velocity=(-1, -3))
Robot(position=(3, 0), velocity=(-1, -2))
Robot(position=(9, 3), velocity=(2, 3))
Robot(position=(7, 3), velocity=(-1, 2))
Robot(position=(2, 4), velocity=(2, -3))
Robot(position=(9, 5), velocity=(-3, -3))


In [3]:
# test case (part 1)
def part_1(reader: InputReader, dimensions=(101, 103), debug=False) -> int:
    robots = domain_from_input(reader)
    time = 100

    # run simulation
    while time > 0:
        for robot in robots:
            robot.move(dimensions)
        time -= 1

    # get positions
    positions = {}
    for robot in robots:
        position = robot.position
        positions[position] = positions.get(position, 0) + 1

    quadrants = [0, 0, 0, 0]

    rows = range(dimensions[1])
    cols = range(dimensions[0])
    middle_row = dimensions[1] // 2
    middle_col = dimensions[0] // 2

    # print the positions in a grid
    for y in rows:
        for x in cols:
            if debug:
                if y == middle_row or x == middle_col:
                    print("X", end="")
                elif (x, y) in positions:
                    print("#", end="")
                else:
                    print(".", end="")

            if y == middle_row or x == middle_col:
                continue

            counts = positions.get((x, y), 0)
            if x > middle_col:
                if y > middle_row:
                    quadrants[0] += counts
                else:
                    quadrants[2] += counts
            else:
                if y > middle_row:
                    quadrants[1] += counts
                else:
                    quadrants[3] += counts

        if debug:
            print()

    if debug:
        for robot in robots:
            print(robot)

    return quadrants[0] * quadrants[1] * quadrants[2] * quadrants[3]


result = part_1(puzzle.example(0), (11, 7), True)
display(result)
assert result == 12

.....X#..#.
.....X.....
#....X.....
XXXXXXXXXXX
.....X.....
...##X.....
.#...X#....
Robot(position=(3, 5), velocity=(3, -3))
Robot(position=(5, 4), velocity=(-1, -3))
Robot(position=(9, 0), velocity=(-1, 2))
Robot(position=(4, 5), velocity=(2, -1))
Robot(position=(1, 6), velocity=(1, 3))
Robot(position=(1, 3), velocity=(-2, -2))
Robot(position=(6, 0), velocity=(-1, -3))
Robot(position=(2, 3), velocity=(-1, -2))
Robot(position=(0, 2), velocity=(2, 3))
Robot(position=(6, 0), velocity=(-1, 2))
Robot(position=(4, 5), velocity=(2, -3))
Robot(position=(6, 6), velocity=(-3, -3))


12

In [66]:
# real case (part 1)
result = part_1(puzzle.input())
display(result)

218619324

In [22]:
def hash_positions(robots):
    positions = {}
    for robot in robots:
        positions[robot.position] = positions.get(robot.position, 0) + 1
    return positions


def tree_candidate(positions, dimensions) -> bool:
    # all the positions should be at most 1
    for _, count in positions.items():
        if count > 1:
            return False
    return True

def print_positions(positions, dimensions):
    rows = range(dimensions[1])
    cols = range(dimensions[0])

    # print the positions in a grid
    for y in rows:
        for x in cols:
            height = positions.get((x, y), 0)
            if height > 0:
                print(f"{height}", end="")
            else:
                print(".", end="")

        print()
    print()

def write_positions(count, positions, dimensions):
    rows = range(dimensions[1])
    cols = range(dimensions[0])

    with open("output.txt", "a") as f:
        f.write(f"Count: {count}\n")

        # print the positions in a grid
        for y in rows:
            for x in cols:
                height = positions.get((x, y), 0)
                if height > 0:
                    f.write(f"*")
                else:
                    f.write(".")

            f.write("\n")
        f.write("\n")

# test case (part 2)
def part_2(reader: InputReader, dimensions=(101, 103), debug=False) -> int:
    robots = domain_from_input(reader)
    limit = 10000
    count = 0
    solution = None
    # delete output.txt
    with open("output.txt", "w") as f:
        f.write("")

    # run simulation
    while count < limit and solution is None:
        count += 1
        for robot in robots:
            robot.move(dimensions)

        positions = hash_positions(robots)
        if tree_candidate(positions, dimensions):
            print(f"Found candidate at count={count}")
            if debug:
                print_positions(positions, dimensions)
                write_positions(count, positions, dimensions)
            solution = count            

    # print the positions in a grid
    if debug:
        for robot in robots:
            print(robot)

    return solution


result = part_2(puzzle.input())
display(result)
assert result == 6446

Found candidate at count=6446


6446

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()